auth.getSession

Fetch a session key for a user. The third step in the authentication process. See the authentication how-to for more information.
Params

token (Required) : A 32-character ASCII hexadecimal MD5 hash returned by step 1 of the authentication process (following the granting of permissions to the application by the user)
api_key (Required) : A Last.fm API key.
api_sig (Required) : A Last.fm method signature. See authentication for more information.

artist.getTopTracks

Get the top tracks by an artist on Last.fm, ordered by popularity
Example URLs

JSON: /2.0/?method=artist.gettoptracks&artist=cher&api_key=YOUR_API_KEY&format=json

XML: /2.0/?method=artist.gettoptracks&artist=cher&api_key=YOUR_API_KEY
Params

artist (Required (unless mbid)] : The artist name
mbid (Optional) : The musicbrainz id for the artist
autocorrect[0|1] (Optional) : Transform misspelled artist names into correct artist names, returning the correct version instead. The corrected artist name will be returned in the response.
page (Optional) : The page number to fetch. Defaults to first page.
limit (Optional) : The number of results to fetch per page. Defaults to 50.
api_key (Required) : A Last.fm API key.

PASO 1: Obtener token de autenticación que necesitaremos para iniciar sesión.

In [112]:
import requests
import pandas as pd

API_KEY = '357e10dfd8c3d43a08795cbb92a9cf24'
API_SECRET = '1702885e042cee4d290d834146c2891a'

url = f'https://ws.audioscrobbler.com/2.0/?method=auth.gettoken&api_key={API_KEY}&format=json'
params = {
    'api_key': API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    token_value = response.text.split(':')[1].strip('}"')  # Guarda el token de la respuesta
    print(f'Tu token es: {token_value}')
else:
    print('Error al obtener el token')

Tu token es: 8h4bIeGCeFAkT0jkWTayPTzN75JF6RZ6


PASO 2: Autorizar el token en la WEB de last.fm.com -- > https://www.last.fm/api/auth?api_key=INTRODUCIR_API_KEY&token=INTRODUCIR_TOKEN:
https://www.last.fm/api/auth?api_key=357e10dfd8c3d43a08795cbb92a9cf24&token=INTRODUCIR_TOKEN

PASO 3: Obtener sesión.
Concatenar los parámetros en el orden correcto para generar el api_sig. El api_sig es una firma que asegura que la solicitud es válida. Se genera concatenando los parámetros en este orden: api_key, method, token, y api_secret.

In [113]:
import requests
import hashlib

# Paso 1: Concatenar los parámetros en el orden correcto para generar el api_sig
concatenated_params1 = f'api_key{API_KEY}methodauth.getSessiontoken{token_value}{API_SECRET}'
api_sig = hashlib.md5(concatenated_params1.encode('utf-8')).hexdigest()

# Paso 2: Preparar los parámetros para la solicitud POST
url3 = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'auth.getSession',
    'api_key': API_KEY,
    'token': token_value,
    'api_sig': api_sig,
    'format': 'json'
}

# Hacer la solicitud POST con los parámetros en el cuerpo de la solicitud
response = requests.post(url3, data=params)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    session_key = response.json()['session']['key']  # Mostrar la respuesta en formato JSON
    print(session_key)
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error si es posible

La llamada es correcta
K9yTrrMFEzcpYl6jYwPcgy8u7hLFOGlZ


In [118]:
import datetime
import time

timestamp_actual = int(time.time())
#fecha_y_hora = datetime.datetime(2023, 10, 22, 12, 0, 0)
#timestamp = int(fecha_y_hora.timestamp())

track = 'Feel Good Inc.'
artist = 'Gorillaz'

url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp_actual,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, params=params)

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')

Error 400: Bad Request


In [39]:
url1 = f'http://www.last.fm/api/auth/?{API_KEY}token={token}'
params = {
    'token': token,
    'api_key': API_KEY,
    #'api_sig': API_SECRET
    }

response = requests.get(url1, params=params)

if response.status_code == 200:
    print(f'la llamada es correcta')
else:
    print(response.status_code)
    print(response.reason)

ConnectionError: HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /api/auth/?357e10dfd8c3d43a08795cbb92a9cf24token=%7B%22token%22:%22l2lkO9D3hVl0gYN9Z096uUvqddhjT6YZ%22%7D&token=%7B%22token%22%3A%22l2lkO9D3hVl0gYN9Z096uUvqddhjT6YZ%22%7D&api_key=357e10dfd8c3d43a08795cbb92a9cf24 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x129f42ac0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [33]:
import hashlib

Error 403: Forbidden


In [ ]:
# Paso 1: Concatenar los parámetros en el orden correcto
concatenated_params = f'api_key{API_KEY}methodauth.getSessiontoken{token}{API_SECRET}'

# Paso 2: Generar el hash MD5 de la cadena concatenada
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Paso 3: Hacer la solicitud con los parámetros correctos
url2 = f'https://ws.audioscrobbler.com/2.0/?method=auth.getSession&api_key={API_KEY}&token={token}&api_sig={api_sig}&format=json'

response = requests.get(url2)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')

In [40]:
# Paso 1: Concatenar los parámetros en el orden correcto para generar el api_sig
concatenated_params1 = f'api_key{API_KEY}methodauth.getSessiontoken{token}{API_SECRET}'

# Paso 2: Generar el hash MD5 de la cadena concatenada
api_sig = hashlib.md5(concatenated_params1.encode('utf-8')).hexdigest()

# Paso 3: Preparar los parámetros para la solicitud POST
url3 = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'auth.getSession',
    'api_key': API_KEY,
    'token': token,
    'api_sig': api_sig,
    'format': 'json'
}

# Hacer la solicitud POST con los parámetros en el cuerpo de la solicitud
response = requests.post(url3, data=params)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error si es posible

Error 403: Forbidden
{"message":"Unauthorized Token - This token has not been issued","error":4}


PRUEBA CON CÓDIGO UNIDO 

In [111]:
import requests
import hashlib
import datetime

# Definición de variables
API_KEY = 'YOUR_API_KEY'  # Reemplaza con tu API Key
API_SECRET = 'YOUR_API_SECRET'  # Reemplaza con tu API Secret
session_key = 'YOUR_SESSION_KEY'  # Reemplaza con tu SESSION_KEY

# Definir la fecha y hora para el timestamp
fecha_y_hora = datetime.datetime(2023, 10, 22, 12, 0, 0)
timestamp = int(fecha_y_hora.timestamp())

# Definir la canción y el artista
track = 'Waka Waka'
artist = 'Shakira'

# Generar api_sig
concatenated_params = (
    f'api_key{API_KEY}'
    f'artist{artist}'
    f'methodtrack.scrobble'
    f'sk{session_key}'
    f'timestamp{timestamp}'
    f'track{track}'
    f'{API_SECRET}'
)
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Preparar los parámetros para la solicitud
url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, data=params)  # Usar `data` en lugar de `params`

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error

Error 403: Forbidden
{"message":"Invalid API key - You must be granted a valid key by last.fm","error":10}


PRUEBA

In [119]:
import requests
import hashlib
import time

# Obtener el timestamp actual
timestamp_actual = int(time.time())

# Información de la canción
track = 'Feel Good Inc.'
artist = 'Gorillaz'

# Generar api_sig
concatenated_params = (
    f'api_key{API_KEY}'
    f'artist{artist}'
    f'methodtrack.scrobble'
    f'sk{session_key}'
    f'timestamp{timestamp_actual}'
    f'track{track}'
    f'{API_SECRET}'  # La API_SECRET se concatena al final
)
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Preparar los parámetros para la solicitud POST
url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp_actual,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, data=params)  # Usar `data` para enviar los parámetros en el cuerpo

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar detalles adicionales del error

Canción scrobbleada correctamente.
{'scrobbles': {'scrobble': {'artist': {'corrected': '0', '#text': 'Gorillaz'}, 'album': {'corrected': '0'}, 'track': {'corrected': '0', '#text': 'Feel Good Inc.'}, 'ignoredMessage': {'code': '0', '#text': ''}, 'albumArtist': {'corrected': '0', '#text': ''}, 'timestamp': '1729718465'}, '@attr': {'ignored': 0, 'accepted': 1}}}


In [125]:
rs = response.json()
rs

{'scrobbles': {'scrobble': {'artist': {'corrected': '0', '#text': 'Gorillaz'},
   'album': {'corrected': '0'},
   'track': {'corrected': '0', '#text': 'Feel Good Inc.'},
   'ignoredMessage': {'code': '0', '#text': ''},
   'albumArtist': {'corrected': '0', '#text': ''},
   'timestamp': '1729718465'},
  '@attr': {'ignored': 0, 'accepted': 1}}}